In [ ]:
import sys
import numpy as np
sys.path.append('..')
from Comms_System import Comms_System
from train_decision_making import train_DM_model
#from DE_minibatch import DE
from DE import DE
from objective_functions import MSE
from objective_functions import crossEntropy
from objective_functions import crossEntropyPytorch
from Network import NeuralNetwork
import matplotlib.pyplot as plt
import scipy.stats
from objective_functions import stablesoftmax as softmax

# Train Model

In [ ]:
symbol_set = [3, 1, -1, -3] # all symbols that we use
num_symbols = 1334
symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
m = 8
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m)
noise_level = 0
# Automatic test
decisions, _, _, _, downsampled = CS.test_CS(noise_level=noise_level, v=False)

X = np.array(downsampled, ndmin=2).T
y = symbol_seq
classes = np.array(symbol_set)
num_classes = len(classes)
sizes = [1, 8, 4]
sizes[-1] = num_classes

# Makes a dictionary that maps a number to each symbol e.g. 3: 0
class_idx = {v: i for i, v in enumerate(classes)}

# Maps class indexes to each value in targets
y = np.array([class_idx[v] for v in y])

# Converts to one-hot-encoded
y = np.eye(num_classes)[y]

# split train-test
splitlen = int(X.shape[0] * 0.75)
Xtrain, ytrain = X[:splitlen], y[:splitlen]
Xtest, ytest = X[splitlen:], y[splitlen:]

In [ ]:
# Inspect symbol distribution in training (downsampled) data

for symbol in symbol_set:

    idx_ = symbol_seq[:splitlen] == symbol
    plt.hist(Xtrain[idx_])

In [ ]:
optimizer = 'DE'
early_stop = False

if optimizer == 'DE':
    D = DE(objective_function=crossEntropy, sizes=sizes, pop_size=50, F=0.55, cr=0.85,
           X=Xtrain, y=ytrain, Xtest=Xtest, ytest=ytest, type='classification', afunc='relu')
    
    if early_stop:
        best_agent = D.early_stop_training(patience=15)
    else:
        best_agent = D.evolution(num_epochs=200, verbose=True, print_epoch=500)
        D.evaluate()
    
elif optimizer == 'BP':
    NN = NeuralNetwork(sizes=sizes, afunc='relu')
    NN.train_loop(Xtrain, ytrain, Xtest, ytest, epochs=1000, batch_size=Xtrain.shape[0], 
                  cost=True, acc=True, plot=True)
    best_agent = NN

# Prediction Accuracy

In [ ]:
# Get predictions
predictions = D.best_agent.feedforward(Xtest).argmax(axis=1)
true = ytest.argmax(axis=1)

# Convert back to symbols
predicted_symbols = classes[predictions]
true_values = classes[true]
# print(predicted_symbols, true_values)
correct_preds = np.equal(true_values, predicted_symbols)
print("Accuracy: ", (sum(correct_preds) / len(true_values)) * 100, "%")

# Error Rate

In [ ]:
symbol_set = [3, 1, -1, -3] # all symbols that we use
num_symbols = 10000
symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)

In [ ]:
for i in range(10):

    m = 8
    CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m)
    sigma = 2

    decisions, NN_decisions, _, _, _ = CS.test_CS(noise_level=sigma, dec_model=D.best_agent, v=False)
    #X = np.array(downsampled, ndmin=2).T

    print('Euclidean Distance Error Rate: {}%'.format((CS.evaluate(decisions)[1]*100).round(2)))
    print('Neural Net Error Rate: {}%'.format((CS.evaluate(NN_decisions)[1]*100).round(2)))

----

# How to retrain automatically

In [ ]:
train = False
if train:
    D = train_DM_model(sizes=[1,8,4], num_symbols=1000, epochs=2001)

In [ ]:
# Evaluate
symbol_set = [3, 1, -1, -3] # all symbols that we use
num_symbols = 10000
symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
m = 8
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m)
noise_level = 1

decisions, _ , _, downsampled = CS.test_CS(noise_level=noise_level, v=False)
X = np.array(downsampled, ndmin=2).T
classes = np.array(symbol_set)
new_NN_decisions = classes[D.best_agent.feedforward(X).argmax(axis=1)]

print('Euclidean Distance Error Rate: {}%'.format((CS.evaluate(decisions)[1]*100).round(2)))
print('Neural Net Error Rate: {}%'.format((CS.evaluate(new_NN_decisions)[1]*100).round(2)))

# Does model perform better? Then save the weights

In [ ]:
save_weights = False

if save_weights:
    D.save_params('decision_making_weights_CE_new3', 'decision_making_biases_CE_new3')